In [3]:
import os
import random
import sys
from PIL import Image, ImageDraw, ImageFont

# --- Image Processing Functions ---

def resize_crop_to_fill(img, size=(512, 512)):
    """
    Crops and resizes an image to fill the target dimensions exactly,
    avoiding stretching or padding by cropping from the center.
    """
    original_size = img.size
    
    # Calculate aspect ratios to determine cropping strategy
    img_ratio = img.width / img.height
    target_ratio = size[0] / size[1]
    
    if img_ratio > target_ratio:
        # Image is wider than the target aspect ratio, so crop the width
        new_height = img.height
        new_width = int(img.height * target_ratio)
        left = (img.width - new_width) // 2
        crop_box = (left, 0, left + new_width, new_height)
    else:
        # Image is taller than or equal to the target, so crop the height
        new_width = img.width
        new_height = int(img.width / target_ratio)
        top = (img.height - new_height) // 2
        crop_box = (0, top, new_width, top + new_height)
    
    # Perform the crop and resize operations
    cropped = img.crop(crop_box)
    resized = cropped.resize(size, Image.Resampling.LANCZOS)
    
    print(f"    - Original: {original_size} → Cropped: {cropped.size} → Final: {resized.size}")
    return resized

def make_collage(images, grid_size=(2, 2), image_size=(512, 512)):
    """
    Creates a collage by arranging a list of images in a grid.
    """
    # Create a new blank canvas for the collage
    collage_width = grid_size[0] * image_size[0]
    collage_height = grid_size[1] * image_size[1]
    canvas = Image.new("RGB", (collage_width, collage_height))
    
    positions = [
        (col * image_size[0], row * image_size[1])
        for row in range(grid_size[1])
        for col in range(grid_size[0])
    ]
    
    print("\n🎨 Creating collage:")
    for i, (img, pos) in enumerate(zip(images, positions)):
        print(f"    - Placing image {i+1} at {pos}")
        canvas.paste(img, pos)
        
    return canvas

def add_text_overlay(canvas, text):
    """
    Adds high-quality, centered text with a black outline to an image.
    Renders text at a high resolution and downsamples for crispness.
    """
    # For anti-aliasing, create a high-resolution canvas (4x) to draw on
    scale_factor = 4
    high_res_size = (canvas.width * scale_factor, canvas.height * scale_factor)
    high_res_canvas = canvas.resize(high_res_size, Image.Resampling.LANCZOS)
    draw = ImageDraw.Draw(high_res_canvas)
    
    # Scale font size and outline thickness accordingly
    font_size = 50 * scale_factor
    outline_thickness = 2 * scale_factor
    
    # Find a suitable font available on the system
    font = find_system_font(font_size)
    
    # Get text dimensions to calculate center position
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    # Calculate centered position and then shift it up slightly (by 8% of canvas height)
    x = (high_res_canvas.width - text_width) // 2
    y = (high_res_canvas.height - text_height) // 2
    y -= int(canvas.height * 0.02) * scale_factor
    
    # Draw the black outline by drawing the text at 8 offset positions
    draw.text((x - outline_thickness, y - outline_thickness), text, font=font, fill="black")
    draw.text((x + outline_thickness, y - outline_thickness), text, font=font, fill="black")
    draw.text((x - outline_thickness, y + outline_thickness), text, font=font, fill="black")
    draw.text((x + outline_thickness, y + outline_thickness), text, font=font, fill="black")
    draw.text((x - outline_thickness, y), text, font=font, fill="black")
    draw.text((x + outline_thickness, y), text, font=font, fill="black")
    draw.text((x, y - outline_thickness), text, font=font, fill="black")
    draw.text((x, y + outline_thickness), text, font=font, fill="black")
    
    # Draw the main white text on top of the outline
    draw.text((x, y), text, font=font, fill="white")
    
    # Downsample the high-resolution canvas to the original size
    final_canvas = high_res_canvas.resize(canvas.size, Image.Resampling.LANCZOS)
    
    print(f"    - Added text overlay: '{text}'")
    return final_canvas

# --- Helper & Utility Functions ---

def find_system_font(size):
    """
    Attempts to find a common, high-quality font across different OSes.
    Falls back to the default Pillow font if none are found.
    """
    font_paths = {
        'darwin': [ # macOS
            "/System/Library/Fonts/SF-Pro-Display-Bold.otf",
            "/System/Library/Fonts/Helvetica.ttc",
        ],
        'win32': [ # Windows
            "C:/Windows/Fonts/Arial.ttf",
            "C:/Windows/Fonts/Verdana.ttf",
        ],
        'linux': [ # Linux
            "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
        ]
    }
    
    # Get platform-specific font paths, or an empty list if OS not listed
    system_paths = font_paths.get(sys.platform, [])
    
    for path in system_paths:
        try:
            return ImageFont.truetype(path, size)
        except IOError:
            continue # Font not found, try the next one
            
    # If no specific fonts were found, fall back to the default
    print("    - Warning: Using default font. For best results, ensure a standard TTF/OTF font is available.")
    return ImageFont.load_default()


def get_user_folder_selection(folders):
    """
    Prompts the user to select which folders to process from a numbered list.
    Supports single numbers, commas, ranges (e.g., "1, 3, 5-7"), and random selection (e.g., "r5").
    """
    print("\n📂 Available hairstyle folders:")
    for i, folder in enumerate(folders, 1):
        print(f"  {i}. {folder}")
    
    print("\nSelect folders to process:")
    print("  - Enter numbers separated by commas (e.g., 1,3,5)")
    print("  - Enter ranges with dashes (e.g., 1-4,7)")
    print("  - Enter 'all' to process all folders")
    print("  - Enter 'rX' to randomly select X folders (e.g., r5)")
    
    while True:
        selection = input("\nYour selection: ").strip().lower()
        
        if selection == 'all':
            return folders
        
        # Check for random selection pattern (rX)
        if selection.startswith('r') and len(selection) > 1:
            try:
                num_random = int(selection[1:])
                if num_random > len(folders):
                    print(f"⚠️  Warning: Requested {num_random} folders but only {len(folders)} available. Using all folders.")
                    return folders
                elif num_random <= 0:
                    print("❌ Number must be greater than 0.")
                    continue
                else:
                    selected_folders = random.sample(folders, num_random)
                    print(f"\n✅ Randomly selected {len(selected_folders)} folders: {', '.join(selected_folders)}")
                    return selected_folders
            except ValueError:
                print("❌ Invalid random selection format. Use 'rX' where X is a number (e.g., r5).")
                continue
        
        try:
            selected_indices = set()
            parts = selection.split(',')
            for part in parts:
                part = part.strip()
                if '-' in part:
                    start, end = map(int, part.split('-'))
                    selected_indices.update(range(start, end + 1))
                elif part:
                    selected_indices.add(int(part))
            
            selected_folders = []
            for idx in sorted(list(selected_indices)):
                if 1 <= idx <= len(folders):
                    selected_folders.append(folders[idx - 1])
                else:
                    print(f"⚠️  Warning: Index {idx} is out of range (1-{len(folders)})")
            
            if selected_folders:
                print(f"\n✅ Selected {len(selected_folders)} folders: {', '.join(selected_folders)}")
                return selected_folders
            else:
                print("❌ No valid folders selected. Please try again.")
        
        except ValueError:
            print("❌ Invalid input format. Please use numbers, commas, dashes, 'all', or 'rX' only.")

# --- Main Execution ---

def main():
    """Main function to run the collage generation process."""
    input_root = "hairstyles"
    output_root = "out"

    if not os.path.isdir(input_root):
        print(f"❌ Error: Input directory '{input_root}' not found.")
        print("Please create it and add subfolders with images.")
        return

    os.makedirs(output_root, exist_ok=True)

    print("🔍 Scanning hairstyle folders...")
    available_folders = sorted([
        folder for folder in os.listdir(input_root)
        if os.path.isdir(os.path.join(input_root, folder))
    ])

    if not available_folders:
        print(f"❌ No subfolders found in the '{input_root}' directory.")
        return

    selected_folders = get_user_folder_selection(available_folders)

    print(f"\n🚀 Starting processing for {len(selected_folders)} selected folders...\n" + "-" * 60)

    for style_folder in selected_folders:
        folder_path = os.path.join(input_root, style_folder)
        print(f"📁 Processing folder: {style_folder}")
        
        image_files = [
            f for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png", ".webp"))
        ]

        if len(image_files) < 4:
            print(f"   - ❌ Skipping: Not enough images found (found {len(image_files)}, need 4).\n")
            continue

        selected_files = random.sample(image_files, 4)
        print(f"   - 🎯 Randomly selected 4 images.")
        
        images = []
        for f in selected_files:
            try:
                img = Image.open(os.path.join(folder_path, f)).convert("RGB")
                processed_img = resize_crop_to_fill(img)
                images.append(processed_img)
            except Exception as e:
                print(f"   - ⚠️ Error processing file {f}: {e}")

        if len(images) < 4:
            print(f"   - ❌ Skipping: Failed to process enough images.\n")
            continue
        
        collage = make_collage(images)
        
        display_name = style_folder.replace("_", " ").replace("-", " ").title()
        collage = add_text_overlay(collage, display_name)
        
        output_path = os.path.join(output_root, f"{style_folder}_collage.png")
        collage.save(output_path, "PNG")
        
        print(f"\n✅ Saved: {output_path}\n" + "-" * 60)

    print("\n🎉 All done! Collages created successfully.")

if __name__ == "__main__":
    main()

🔍 Scanning hairstyle folders...

📂 Available hairstyle folders:
  1. buzz_cut
  2. comma_cut
  3. crew_cut
  4. jay_jo_cut
  5. low_fade
  6. low_taper
  7. mid_part
  8. mod_cut
  9. modern_mullet
  10. textured_crop
  11. two_block
  12. warrior_cut
  13. wolf_cut

Select folders to process:
  - Enter numbers separated by commas (e.g., 1,3,5)
  - Enter ranges with dashes (e.g., 1-4,7)
  - Enter 'all' to process all folders
  - Enter 'rX' to randomly select X folders (e.g., r5)

Your selection: r7

✅ Randomly selected 7 folders: mid_part, two_block, comma_cut, buzz_cut, low_fade, jay_jo_cut, low_taper

🚀 Starting processing for 7 selected folders...
------------------------------------------------------------
📁 Processing folder: mid_part
   - 🎯 Randomly selected 4 images.
    - Original: (814, 802) → Cropped: (802, 802) → Final: (512, 512)
    - Original: (1086, 1128) → Cropped: (1086, 1086) → Final: (512, 512)
    - Original: (694, 692) → Cropped: (692, 692) → Final: (512, 512)
    